# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA is used as the PEFT technique. LoRA is the only PEFT technique that is compatible with all models at this time.

* Model: GPT-2 is selected as the model. This is a relatively small model that is compatible with sequence classification and LoRA.

* Evaluation approach:  "evaluate" method with a Hugging Face Trainer is used as the evlaution approach

* Fine-tuning dataset: IMDB dataset, which contains movie reviews

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
!pip install -U datasets
! pip install -q "datasets==2.15.0"

Defaulting to user installation because normal site-packages is not writeable
  Using cached datasets-3.3.2-py3-none-any.whl (485 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.15.0
    Uninstalling datasets-2.15.0:
      Successfully uninstalled datasets-2.15.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Load and preprocess a dataset
# Import the datasets: use the IMDB movie review dataset to train and evaluate your model. 
# The IMDB dataset contains movie reviews that are labeled as either positive or negative.
from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Show the dataset
#ds

#ds["test"][3]

In [3]:
splits[0][1:10]

'rain'

In [4]:
# Load a pretrained HF model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2", 
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

for param in model.base_model.parameters():
    param.requires_grad = False

    
print(model)

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


In [5]:
#tokenizing Text
from transformers import AutoTokenizer, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("gpt2", max_length=1024)
config = AutoConfig.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id=50256



def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples["text"], truncation=True)

'''
print(ds["test"][1])
print("...")
print(tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger."))
print("...")
print(ds["test"][3])
print("...")
print(preprocess_function(ds["test"][3]))
'''
print(splits)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)   
    

['train', 'test']


/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
#tokenized_ds['train'][1]['input_ids']

In [7]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

#Evaluate model before finetuning
trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.7127957344055176,
 'eval_accuracy': 0.508,
 'eval_runtime': 16.4011,
 'eval_samples_per_second': 30.486,
 'eval_steps_per_second': 7.621}

In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.538604,0.730000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=125, training_loss=0.7189378662109375, metrics={'train_runtime': 41.6694, 'train_samples_per_second': 11.999, 'train_steps_per_second': 3.0, 'total_flos': 136488765947904.0, 'train_loss': 0.7189378662109375, 'epoch': 1.0})

In [9]:
# Evaluating the model after finetuning
trainer.evaluate()

{'eval_loss': 0.5386037826538086,
 'eval_accuracy': 0.73,
 'eval_runtime': 16.5633,
 'eval_samples_per_second': 30.187,
 'eval_steps_per_second': 7.547,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(task_type="SEQ_CLS", 
                    inference_mode=False, 
                    r=8, 
                    lora_alpha=32, 
                    lora_dropout=0.1
)

lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 297,984 || all params: 124,737,792 || trainable%: 0.23888830740245906


In [11]:
#training the LoRA model
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer_l = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis_lora",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

trainer_l.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.371011,0.866000


Checkpoint destination directory ./data/sentiment_analysis_lora/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=125, training_loss=0.6184431762695313, metrics={'train_runtime': 67.0252, 'train_samples_per_second': 7.46, 'train_steps_per_second': 1.865, 'total_flos': 136964465381376.0, 'train_loss': 0.6184431762695313, 'epoch': 1.0})

In [12]:
trainer_l.evaluate()

{'eval_loss': 0.37101083993911743,
 'eval_accuracy': 0.866,
 'eval_runtime': 17.0905,
 'eval_samples_per_second': 29.256,
 'eval_steps_per_second': 7.314,
 'epoch': 1.0}

In [13]:
lora_model.save_pretrained("gpt-lora")
tokenizer.save_pretrained("gpt2_lora")
config.save_pretrained('gpt2_lora')


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [14]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")

#tokenizer = AutoTokenizer.from_pretrained("gpt2_lora")

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
#tokenizing Text
from transformers import AutoTokenizer, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("gpt2")
config = AutoConfig.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
lora_model.config.pad_token_id = tokenizer.pad_token_id

def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    return tokenizer(examples["text"], truncation=True)

'''
print(ds["test"][1])
print("...")
print(tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger."))
print("...")
print(ds["test"][3])
print("...")
print(preprocess_function(ds["test"][3]))
'''
print(splits)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)  

['train', 'test']


In [24]:
#training the LoRA model
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer_l = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis_lora",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)


In [25]:
trainer_l.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.37101083993911743,
 'eval_accuracy': 0.866,
 'eval_runtime': 16.7691,
 'eval_samples_per_second': 29.817,
 'eval_steps_per_second': 7.454}